# Crystal Cascade
A Sierra Nevada/Little Creatures inspired American Pale Ale

## Recipe

### Malt Bill:
 - Pilsner Malt - 2 Row (?EBC)
 - Australian Ale Malt - 2 Row (?EBC)
 - Crystal Malt (?EBC)

### Hops:
 - Cascade (?Alpha Acids)

Make a widget window for all the inputs

In [1]:
from ipywidgets import interact, fixed
import ipywidgets as widgets

In [27]:
?malt_mass_box.style

In [58]:
def make_widget(value, description):
    style = {'description_width' : '50%'}
    return widgets.FloatText(value=value, description=description, style=style)


batch_size_box = make_widget(5, 'Batch Size (L) =')
malt_mass_box = make_widget(1, 'Total Malt Mass (kg) =')

words = ['correct', 'horse', 'battery', 'staple']
items = [widgets.FloatText(description=w) for w in words]
left_box = widgets.VBox([batch_size_box,
                         malt_mass_box])
mid_box = widgets.VBox([items[0],
                        items[1]])
right_box = widgets.VBox([items[2],
                          items[3]])
widgets.HBox([left_box, mid_box, right_box])

### Calculate Water amounts
Need functions to take in batch size and work out the amounts all the way to strike water, inculding assumptions for losses

In [65]:
batch_size = batch_size_box.value
print(batch_size, 'L')

5.0 L


In [ ]:
# This will be a module (file)
class constants(object):
    # Approximate rate of evaporation at boil (L/hr)
    evap_rate = ????????????
    # Approximate loss of volume due to grain absorption
    grain_disp = ????????????
    # Efficiency of getting sugars from malt
    eff = ?????????

In [ ]:
def strike_water():
    pass
def pre_boil_volume():
    pass
def post_boil_volume():
    return pre_boil_volume() - evaporation(time)
def evaporation(time=1):
    """ Returns evaporation (L) for a given boil time (hrs) """
    return time * constants.evap_rate

### Calculate Expected Original Gravity

In [66]:
malt_mass = malt_mass_box.value
print(batch_size, 'L;', malt_mass, 'kg')

5.0 L; 1.25 kg
